*Setup*

In [1]:
import pyodbc
# pyodbc is an open source Python module that makes accessing ODBC databases simple
import pandas as pd
# pandas is a Python package/module(?) that provides fast, flexible, and expressive data structures designed to make working with "relational" or "labeled" data both easy and intuitive
# pandas is an open source data analysis library built on top of the Python programming language
# The as pd portion of the code then tells Python to give pandas the alias of pd. This allows you to use pandas functions by simply typing pd.function_name rather than pandas.function_name.
import getpass
# password input

In [2]:
# Specify database connection parameters
server = "ifw9ecos-bvdb11.fws.doi.net\ecos_beta"
database = "IRIS_DataWarehouse"

# Uncomment if not using Windows authentication
username = input()
# The input() function allows user input. Program flow will be stopped until the user has given an input
password = getpass.getpass()
# The getpass() function of getpass prompts the user to enter a password. By default the keys entered by user in the terminal are not echoed.

TempLogin4KhemSo
········


In [3]:
##----------------------------------------------------
# Create a database connection object using a Data Source Name (DSN)
# defined by the Windows ODBC Data Source Administrator
##----------------------------------------------------
#dsn_name = ""
#db_connection = pyodbc.connect("DSN="+dsn_name+";UID="+username+";PWD="+password)

##----------------------------------------------------
#  Create a database connection object by specifying connection parameters
##----------------------------------------------------

# Uncomment if not using Windows authentication
db_connection = pyodbc.connect("\
    DRIVER={SQL Server};\
    SERVER=" + server + ";\
    PORT=1443;\
    DATABASE=" + database + ";\
    UID=" + username + ";\
    PWD=" + password)

# connect Python to SQL Server
# connect Python to SQL Server
# connecting to SQL using pyodbc, use connect
# The PYODBC connection string, if connecting outside of the network the SQL Server is on, requires a port number. Simply add 'PORT=1433
# https://datatofish.com/how-to-connect-python-to-sql-server-using-pyodbc/

#db_connection = pyodbc.connect("\
#    DRIVER={SQL Server};\
#    SERVER=" + server + ";\
#    PORT=1443;\
#    DATABASE=" + database + ";\
#    Trusted_Connection=yes")



In [4]:
cursor = db_connection.cursor()
#The cursor object allows us to execute queries and retrieve rows. The cursor object is an instance of MySQLCursor class.
#A cursor is an object which helps to execute the query and fetch the records from the database.
#Cursors are simply objects used for executing SQL commands and retrieving the results.

#Python is an object-oriented programming language. Everything is in Python treated as an object, including variable, function, list, tuple, dictionary, set, etc

*Create mapper df*

In [145]:
#check wd
#import os
#print(os.getcwd())

#read in the "mapper file" * this only includes fields with a one-to-one relationship 
mapper = pd.read_csv('one_to_one.csv', header=0)
mapper

,Source,Node,Path,Parent,Relationship
0,SurveyName,title,metadata.idinfo.citation.citeinfo.title.text,metadata.idinfo.citation.citeinfo,one-to-one
1,Station,address,metadata.distinfo.distrib.cntinfo.cntaddr.addr...,metadata.distinfo.distrib.cntinfo.cntaddr,one-to-one


In [6]:
##TO DO: iterate through all variables in the source_fields list below (aka loop through all fields in the "source column")
source_fields = mapper['Source'].tolist()

variable = source_fields[0]

mapper_select = mapper.query('Source == @variable')
mapper_select

,Source,Node,Path,Parent,Relationship
0,SurveyName,title,metadata.idinfo.citation.citeinfo.title.text,metadata.idinfo.citation.citeinfo,one-to-one


In [ ]:
#mapper_select.loc[0, "Path"]
#mapper_select.loc[0, "Node"]

*Create Queries*

In [7]:
PRIMRsurveyID = 'FF01RORG00-072'
#print(PRIMRsurveyID)

queryIdentification = "SELECT [" + variable + "] FROM [IRIS_DataWarehouse].[Refuges].[VW_Surveys] WHERE [SurveyCode] = '" + PRIMRsurveyID + "';"
dfIdentification = pd.read_sql(queryIdentification, db_connection)
dfIdentification

,SurveyName
0,Tufted Puffin - Breeding Distribution and Abun...


In [115]:
from pymdwizard.core.xml_utils import XMLRecord, XMLNode #import in this manner requires adding a path file to your site-packages directory
# import nodes

test = "C:/Users/lgoetze/scripting/Git/Git/PRIMR_to_FGDC/PRIMR_Metadata_Template_20220429.xml"
#set WD to metadata template
template_md = XMLRecord(test)
#load XML
#template_md 
# view XML

METHOD #1

In [144]:
template_md.metadata.idinfo.citation.citeinfo.title.text = dfIdentification.loc[0,variable]
template_md.metadata.idinfo.citation.citeinfo.lworkcit.citeinfo = dfIdentification.loc[0,variable]
## WORK ON THIS. How to pull path from ""template_md." + mapper_select.loc[0, "Path"]"
#template_md

#template_md.metadata.search_xpath("idinfo/citation/citeinfo/title")
#template_md.metadata.idinfo.citation.citeinfo.lworkcit.citeinfo 

<title>Tufted Puffin - Breeding Distribution and Abundance</title>

In [150]:
path ="template_md"
path
locals()[path]

#path2 ="template_md." + mapper_select.loc[0, "Path"]
#path2
#locals()[path2]


#locals()[path].metadata.idinfo

## STUCK HERE

<metadata>
  <idinfo>
    <citation>
      <citeinfo>
        <origin>SurveyCoordinatorName from Refuges.DimSurvey</origin>
        <origin>Other Contact Name [not available in IRIS warehouse]</origin>
        <pubdate>
        </pubdate>
        <title>Tufted Puffin - Breeding Distribution and Abundance</title>
        <edition>PRIMRproduct_v1 [what is this?]</edition>
        <geoform>
        </geoform>
        <pubinfo>
          <pubplace>CityName, StateName from DimCity</pubplace>
          <publish>U.S. Fish and Wildlife Service</publish>
        </pubinfo>
        <onlink>URL to ServCat Survey Products [not available in IRIS warehouse]</onlink>
        <lworkcit>
          <citeinfo>
            <origin>SurveyCoordinatorName from Refuges.DimSurvey</origin>
            <pubdate>
            </pubdate>
            <title>SurveyName from Refuges.DimSurvey</title>
            <edition>PRIMRproduct_v1 [what is this?]</edition>
            <geoform>
            </geoform>
           

METHOD #2

In [17]:
import lxml
from lxml import etree as et

def createXMLNode(df, element_tag):
    
    element = et.Element(element_tag)
    element.text = dfIdentification.loc[0,variable]
      
    
    return(XMLNode(et.tostring(element, pretty_print=True).decode('utf-8')))

new_node = createXMLNode(dfIdentification, mapper_select.loc[0, "Node"])
new_node

<title>Tufted Puffin - Breeding Distribution and Abundance</title>

In [35]:
node = template_md.metadata.idinfo.citation.citeinfo
#node = template_md.metadata.idinfo.citation.citeinfo.lworkcit.citeinfo
node.replace_child(new_node)
node


<citeinfo>
  <origin>SurveyCoordinatorName from Refuges.DimSurvey</origin>
  <origin>Other Contact Name [not available in IRIS warehouse]</origin>
  <pubdate>
  </pubdate>
  <title>Tufted Puffin - Breeding Distribution and Abundance</title>
  <edition>PRIMRproduct_v1 [what is this?]</edition>
  <geoform>
  </geoform>
  <pubinfo>
    <pubplace>CityName, StateName from DimCity</pubplace>
    <publish>U.S. Fish and Wildlife Service</publish>
  </pubinfo>
  <onlink>URL to ServCat Survey Products [not available in IRIS warehouse]</onlink>
  <lworkcit>
    <citeinfo>
      <origin>SurveyCoordinatorName from Refuges.DimSurvey</origin>
      <pubdate>
      </pubdate>
      <title>Tufted Puffin - Breeding Distribution and Abundance</title>
      <edition>PRIMRproduct_v1 [what is this?]</edition>
      <geoform>
      </geoform>
      <pubinfo>
        <pubplace>CityName, StateName from DimCity</pubplace>
        <publish>U.S. Fish and Wildlife Service</publish>
      </pubinfo>
    </citeinfo>

SCRATCH BELOW

In [ ]:
path = mapper_select.loc[0, "Path"]
final = str("template_md." + path)
kw = locals()[final]
kw

In [ ]:
test = ".metadata.idinfo.citation.citeinfo.title.text"

#locals()[" + test + "]

In [ ]:
#template_md.metadata.idinfo.citation.citeinfo.title.text = dfIdentification.at[0, one_1.iloc[0, 0]]

dfIdentification.loc[0,variable]
#template_md.metadata.idinfo.citation.citeinfo.title.text = dfIdentification[variable]

In [ ]:
template_md

In [ ]:
x = "metadata.idinfo"
# print(f"{x}")
x

text_01 = """print('.metadata')"""
#exec(text_01)

locals()["template_md"].metadata

In [ ]:
template_md.metadata.idinfo.citation.citeinfo.title.text = dfIdentification.at[0,'SurveyName']

In [ ]:

getattr(template_md, metadata)

In [ ]:
getattr(template_md, metadata.idinfo.citation.citeinfo.title.text)

In [ ]:
final = dfIdentification.at[0,'SurveyName']
template_md
final

In [ ]:
value = dfIdentification.at[0,one.iloc[0, 0]]
value

In [ ]:
path = "template_md." + one.iloc[0, 2]
locals()['path'] = 2


In [ ]:
template_md?

In [ ]:
"template_md." + one.iloc[0, 2]


In [ ]:
"template_md." + one.iloc[0, 2]


In [ ]:
template_md

In [ ]:
"template_md." + one.iloc[0, 2] = dfIdentification.at[0,one.iloc[0, 0]]

In [ ]:
queryIdentification = "SELECT [SurveyCode],[Station],[SurveyName], [SurveyType],[ResourceThemeLevel1],[ResourceThemeLevel2] ,[AnswerQ1],[AnswerQ2],[AnswerQ3],[AnswerQ4],[SurveyCoordinatorName],[SurveyCoordinatorTitle],[StationName],[SurveyArea],[ManagementUnits] FROM [IRIS_DataWarehouse].[Refuges].[VW_Surveys] WHERE [SurveyCode] = '" + PRIMRsurveyID + "';"
dfIdentification = pd.read_sql(queryIdentification, db_connection)
dfIdentification

#creates a pandas df?
#Pandas read_sql() method allows you to run any SQL queries from the database
#pandas.read_sql(sql, con, index_col=None)
#sql: It is a SQL query you want to perform on the database.
#con: Database connection.
#https://www.datasciencelearner.com/pandas-read_sql-implementation-examples/#:~:text=Steps%20to%20implement%20Pandas%20read_sql%20%28%29%20method%201,5%20Implement%20the%20pandas%20read_sql%20%28%29%20method.%20

In [ ]:
queryKeywords = "SELECT Refuges.DimKeyword.Name as theme FROM Refuges.DimKeyword RIGHT OUTER JOIN Refuges.DimSurvey INNER JOIN Refuges.FactSurveys ON Refuges.DimSurvey.ID = Refuges.FactSurveys.SurveyID INNER JOIN Refuges.FactSurveyKeywords ON Refuges.FactSurveys.SurveyID = Refuges.FactSurveyKeywords.SurveyID ON Refuges.DimKeyword.ID = Refuges.FactSurveyKeywords.KeywordID WHERE Refuges.DimSurvey.SurveyCode = '" + PRIMRsurveyID + "';"
# query
dfKeywords = pd.read_sql(queryKeywords, db_connection)
# creates a dataframe
#pandas.read_sql reads SQL query or database table into a DataFrame
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html
# pd.read_sql(sql, con)
# sql = SQL query to be executed 
# con = SQLAlchemy connectable, str, or sqlite3 connection
dfKeywords
#dfKeywords.to_csv("keywords.csv")

In [ ]:
querySurveyCoordinator = "SELECT Refuges.DimSurvey.SurveyCode, Refuges.DimSurvey.SurveyCoordinatorName, DimStaff.Title, DimStaff.OrganizationCode, DimStaff.City, DimStaff.StateCode, DimStaff.ZipCode, DimStaff.PhoneNumber, DimStaff.CellNumber, DimStaff.Email, DimStaff.Office, DimStaff.Department, DimOrganization.OrgCode FROM FactStaff INNER JOIN DimStaff ON FactStaff.StaffID = DimStaff.ID AND FactStaff.StaffID = DimStaff.ID AND FactStaff.StaffID = DimStaff.ID INNER JOIN DimOrganization ON FactStaff.OrganizationID = DimOrganization.ID AND FactStaff.OrganizationID = DimOrganization.ID AND FactStaff.OrganizationID = DimOrganization.ID RIGHT OUTER JOIN Refuges.DimSurvey ON DimStaff.FullName = Refuges.DimSurvey.SurveyCoordinatorName WHERE Refuges.DimSurvey.SurveyCode = '" + PRIMRsurveyID + "';"
#create query
dfSurveyCoordinator = pd.read_sql(querySurveyCoordinator, db_connection)
# create a dataframe. pandas.read_sql reads SQL query into a DataFrame
dfSurveyCoordinator
# view dataframe
# dfSurveyCoordinator.to_csv("surveycoordinator.csv")
# save as csv

In [ ]:
queryBioticGroup = "SELECT Refuges.DimSurvey.SurveyCode, Refuges.DimBioticGroup.ITIS_TSN, Refuges.DimBioticGroup.ITIS_TSN_Parent, Refuges.DimBioticGroup.ScientificName_Parent, Refuges.DimBioticGroup.CommonName_Parent,     Refuges.DimBioticGroup.Rank_Parent FROM Refuges.FactSurveyBioticGroups INNER JOIN Refuges.DimSurvey ON Refuges.FactSurveyBioticGroups.SurveyID = Refuges.DimSurvey.ID INNER JOIN Refuges.DimBioticGroup ON Refuges.FactSurveyBioticGroups.BioticGroupID = Refuges.DimBioticGroup.ID WHERE Refuges.DimSurvey.SurveyCode = '" + PRIMRsurveyID + "';"
dfBioticGroup = pd.read_sql(queryBioticGroup, db_connection)
dfBioticGroup
#dfBioticGroup.to_csv("bioticgroup.csv")

*Working with metadata XML*

In [ ]:
from pymdwizard.core.xml_utils import XMLRecord, XMLNode #import in this manner requires adding a path file to your site-packages directory
# import nodes

test = "C:/Users/lgoetze/scripting/Git/Git/PRIMR_to_FGDC/PRIMR_Metadata_Template_20220429.xml"
#set WD to metadata template
original_md = XMLRecord(test)
#load XML
original_md 
# view XML

In [ ]:
# one-to-one is easy
# edit individual node text
original_md.metadata.idinfo.citation.citeinfo.title.text = dfIdentification.at[0,'SurveyName']
# set title 'text' node in XML to "survey name" value in 0 row (first row) of dfIdentification pandas dataframe
original_md.metadata.idinfo.ptcontac.cntinfo.cntorgp.cntper.text = dfSurveyCoordinator.at[0,'SurveyCoordinatorName']
original_md.metadata.idinfo.ptcontac.cntinfo.cntpos.text = dfSurveyCoordinator.at[0,'Title']

In [ ]:
#original_md

In [ ]:
# gather keywords from a variety of queries and append into one dataframe
dfKeywordsSet = (
    pd.DataFrame(dfIdentification)
    .loc[:, ['ResourceThemeLevel1', 'ResourceThemeLevel2', 'SurveyType', 'SurveyCode']]
    .melt()
#melt () function is used to unpivot a given DataFrame from wide format to long format
    .drop(columns=['variable'])
# remove the 'variable' columns because the values will all fall under a 'theme' column
    .rename(columns={'value':'theme'})
# rename column
    .append(dfKeywords)
# append keywords from the 'dfkeywords' dataframe
    .append((
        pd.DataFrame(dfBioticGroup)
        .loc[:, ['ScientificName_Parent']]
        .melt()
        .drop(columns=['variable'])
        .rename(columns={'value':'theme'})),
        ignore_index=True
    )
# append 'dfBioticGroup'
)
dfKeywordsSet

In [ ]:
dfKeywordsSet.iloc[0][0]
#call cell values from "dfKeywordsSet" df (0 row, 0 column)

In [ ]:
kw = original_md.metadata.idinfo.keywords
#assign kw to node of intereset (keyword)
kw.clear_children('theme')
#clear theme node
kw

*Leana testing how to create xml using lxml*

In [ ]:
#dfKeywordsSet.columns.format()
#dfKeywordsSet.index.format()

In [ ]:
# from lxml import etree
# root = etree.XML('<root><a><b/></a></root>')
# etree.tostring(root)
# print(etree.tostring(root, xml_declaration=True))
# print(etree.tostring(root, encoding='iso-8859-1'))
# print(etree.tostring(root, pretty_print=True).decode())

In [ ]:
# from lxml import etree

# root = etree.Element('theme')
# etree.SubElement(root, 'themekt').text = 'themekt_test'
# etree.SubElement(root, 'themekey').text = 'themekey_test'
# # Be mindful when adding assigning variables after
# # the .text on SubElement call, as it will return 
# # the text and not the Element
# #etree.SubElement(inventory, 'themekt', {'count': '5'}).text = 'Golden Delicious'
# #employees = etree.SubElement(store, 'themekey')
# #etree.SubElement(employees, 'SearingFrost', {'title': 'writer'})

# XMLNode(etree.tostring(root, pretty_print=True).decode())

In [ ]:
import lxml
from lxml import etree as et

def createXMLNode_from_df_LG(df, element_tag, subelement_tag, optional_subelement_tag, optional_subelement_text):
    
    element = et.Element(element_tag)
    et.SubElement(element, optional_subelement_tag).text = optional_subelement_text
# if there are ever more than one <optional_subelement_tag> than we should add it to the loop below
    
    for row in df.iterrows():
        subelement = et.SubElement(element, subelement_tag)
        subelement.text = str(row[1][0])

    return(XMLNode(et.tostring(element, pretty_print=True).decode('utf-8')))

new_node = createXMLNode_from_df_LG(dfKeywordsSet, "theme", "themekey", "themkt", "Add 'themeky' text here")

# Resource: https://stackoverflow.com/questions/49439008/creating-an-xml-file-from-a-dataframe-using-python-and-lxml

In [ ]:
new_node

In [ ]:
kw.add_child(new_node)
kw

In [ ]:
original_md.save(fname='PRIMR_Metadata_Template_20220429Testing_lg.xml')

*Khem's code*

In [ ]:
def createXMLNode_from_df(df, subelement_tag, optional_element_tag):
# “def” is short for the word “define” and it is used to signify the beginning of a function definition in Python
# createXMLNode_from_df() is the function
# specify df, subelement tag, and optional element tag
    xml_data = ['<root>']
# create starting point
    for column in df.columns:
        xml_data.append('<{}>'.format(column))# Opening element tag
# adds the "theme" node"
        xml_data.append(optional_element_tag.format(column))
# adds the "themekt" node
        for field in df.index:
# for every row in df.index
# writing sub-elements
            xml_data.append('<temp_tag>{1}</temp_tag>'.format(field, df[column][field]))
# append creating <temp_tag>
        xml_data.append('</{}>'.format(column))  # Closing element tag
# close theme
    xml_data.append('</root>')
# close root

    my_string = ""
    for i in xml_data:
        my_string += ('{}').format(i)

    new_string = (
        my_string
            .replace('<root>','')
            .replace('</root>','')
        # remove root node
            .replace('temp_tag',subelement_tag)
        # replace <temp_tag> with <themekey>
    )
    return(XMLNode(new_string))

newKW = createXMLNode_from_df(dfKeywordsSet, "themekey", "<themekt>FWS-PRIMR</themekt>")
newKW


# lxml generate xml from table
# parse and create xml
# for this node in the metadata
# run ad hoc query and import into access database- database has all relatinships set up
# store mapping in a dataframe
# if we provide the mapping then create a generic functions that you can call

In [ ]:
kw = original_md.metadata.idinfo.keywords
#assign kw to node of intereset (keyword)
kw.clear_children('theme')
kw.add_child(newKW)
kw
# add child

In [ ]:
# write to new xml
original_md.save(fname='PRIMR_Metadata_Template_20220429Testing.xml')
# save